###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2018 D. Koehn, notebook style sheet by L.A. Barba, N.C. Clementi

In [1]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../../style/custom.css'
HTML(open(css_file, "r").read())

# Isotropic elastic media

In the last chapter we derived the equations of motion for wave propagation in a general anisotropic medium. Here, we will derive the isotropic elastic case.

## Stress-Strain relation for isotropic elastic media

The equations of motion for a general anisotropic medium consist of the conservation of momentum  

\begin{equation}
\rho \frac{\partial^2 u_i}{\partial t^2} = \sum_{j=1}^{3} \frac{\partial}{\partial x_j} \sigma_{ij} + f_i
\end{equation}

and the stress-strain relationship

\begin{equation}
\sigma_{ij} = \sum_{k=1}^{3}\sum_{l=1}^{3} c_{ijkl}\; \epsilon_{kl}
\end{equation}

To introduce isotropy we only have to modify the elastic tensor $c_{ijkl}$, so eq.(1) is unaffected. In an isotropic body, which means uniformity in all directions, only 2 elastic constants are independent and the stiffness tensor $c_{ijkl}$ takes the form (Hodge 1961, Jeffreys 1969):

\begin{equation}
c_{ijkl}=\delta_{ij} \delta_{kl} \lambda + (\delta_{ik} \delta_{jl} + \delta_{il} \delta_{jk}) \mu
\end{equation}

where

\begin{align}
\lambda\; \text{and}\; \mu:\; &\text{Lamé parameters} \nonumber \\ 
\delta_{ij}=
\begin{cases}
1 & \rm{\text{if } i = j}\\
0 & \rm{\text{if } i \ne j}
\end{cases} : \;&\text{Kronecker's delta} \nonumber
\end{align}

To calculate the stress-strain relation for an isotropic elastic medium, we have to insert eq. (3) into (2). In the class we have invested some time to calculate $\sigma_{13}$ manually. Alternatively, we can use the symbolic math library **SymPy**. To install SymPy you have to open a terminal and type

``conda install sympy`` 

In [2]:
# import SymPy
from sympy import *

In [3]:
# test symbolic KroneckerDelta function
KroneckerDelta(1,2)

0

In [4]:
# estimate stress-strain relation for isotropic elastic medium 
# using given indices i,j
def stress_strain_iso(i,j):
    
    # shift indices by -1
    i-=1
    j-=1
    
    # define cijkl for isotropic medium
    c_1 = KroneckerDelta(i,j)*KroneckerDelta(k,l)
    c_2 = KroneckerDelta(i,k)*KroneckerDelta(j,l)
    c_3 = KroneckerDelta(i,l)*KroneckerDelta(j,k)
    cijkl = c_1 * lam + (c_2 + c_3) * mu

    # sum over l-index
    sum_l = Sum(cijkl*e[k,l], (l, 0, 2)).doit()

    # sum over k-index to estimate stress
    stress = Sum(sum_l, (k, 0, 2)).doit()
    
    return stress

In [5]:
# define indices
i, j, k, l = symbols('i j k l', integer=True)

# define components of the deformation tensor as symbols
e11, e12, e13, e22, e23, e33 = symbols('e11 e12 e13 e22 e23 e33')

# define material parameters
lam, mu = symbols('lambda mu')

# define deformation tensor
e = Matrix([[e11,e12,e13], [e12,e22,e23], [e13,e23,e33]])

In [6]:
# stress-strain relation
stress = stress_strain_iso(1,3)
print(latex(stress))

2 e_{13} \mu


This was a nice introduction to the computing capabilities of **SymPy**. However, instead of rushing to evaluate the stress-strain relation for an isotropic medium by explicitly calculating the sums over the Kronecker Deltas for each stress component, let's take a closer look again, what we have to evaluate:

\begin{equation}
\sigma_{ij} = \sum_{k=1}^{3}\sum_{l=1}^{3} \{\delta_{ij} \delta_{kl} \lambda + (\delta_{ik} \delta_{jl} + \delta_{il} \delta_{jk}) \mu\} \epsilon_{kl} \nonumber
\end{equation}

First, we rewrite the relation

\begin{equation}
\sigma_{ij} = \sum_{k=1}^{3}\biggl\{\sum_{l=1}^{3} \delta_{ij} \delta_{kl} \epsilon_{kl} \lambda + \mu \sum_{l=1}^{3} \delta_{ik} \delta_{jl} \epsilon_{kl} + \mu \sum_{l=1}^{3} \delta_{il} \delta_{jk} \epsilon_{kl} \biggr\}\nonumber
\end{equation}

and evaluate the sums over all l-indices. The term $\delta_{ij} \delta_{kl} \epsilon_{kl} \lambda$ is only non-zero if $k=l$, so we get:

\begin{equation}
\sigma_{ij} = \sum_{k=1}^{3}\biggl\{\delta_{ij} \epsilon_{kk} \lambda + \mu \sum_{l=1}^{3} \delta_{ik} \delta_{jl} \epsilon_{kl} + \mu \sum_{l=1}^{3} \delta_{il} \delta_{jk} \epsilon_{kl} \biggr\}\nonumber
\end{equation}

The term $\delta_{ik} \delta_{jl} \epsilon_{kl}$ is only non-zero if $l=j$:

\begin{equation}
\sigma_{ij} = \sum_{k=1}^{3}\biggl\{\delta_{ij} \epsilon_{kk} \lambda + \mu \delta_{ik} \epsilon_{kj} + \mu \sum_{l=1}^{3} \delta_{il} \delta_{jk} \epsilon_{kl} \biggr\}\nonumber
\end{equation}

The opposite is true for the term $\delta_{il} \delta_{jk} \epsilon_{kl}$, where only $l=i$ contributes to the summation:

\begin{equation}
\sigma_{ij} = \sum_{k=1}^{3}\biggl\{\delta_{ij} \epsilon_{kk} \lambda + \mu \delta_{ik} \epsilon_{kj} + \mu \delta_{jk} \epsilon_{ki} \biggr\}\nonumber
\end{equation}

Next, we can evaluate the summation over the k-index, where the term $\mu \delta_{ik} \epsilon_{kj}$ is only non-zero for $k=i$ and $\mu \delta_{jk} \epsilon_{ki}$ only non-zero for $k=j$, leading to 

\begin{equation}
\sigma_{ij} = \delta_{ij} \lambda \biggl\{\sum_{k=1}^{3} \epsilon_{kk} \biggr\} + \mu  \epsilon_{ij} + \mu \epsilon_{ji} \nonumber
\end{equation}

Due to the symmetry of the strain-tensor $\epsilon_{ij} = \epsilon_{ji}$, we get

\begin{equation}
\sigma_{ij} = \delta_{ij} \lambda \biggl\{\sum_{k=1}^{3} \epsilon_{kk} \biggr\} + 2 \mu \epsilon_{ij} \nonumber
\end{equation}

And finally, the general **stress-strain relation for an isotropic elastic medium** as 

\begin{equation}
\sigma_{ij}=\lambda \theta \delta_{ij} + 2 \mu \epsilon_{ij} \nonumber
\end{equation}

with the cubic dilatation

\begin{equation}
\theta = tr(\epsilon_{ij}) =  \epsilon_{11} + \epsilon_{22} + \epsilon_{33} = \frac{\partial u_1}{\partial x_1} + \frac{\partial u_2}{\partial x_2} + \frac{\partial u_3}{\partial x_3} \nonumber
\end{equation}


## 3D equations of motion for isotropic elastic medium

Writing out the momentum conservation and stress-strain relation for the isotropic elastic medium, we get

\begin{align}
\rho\frac{\partial^2 u_1}{\partial t^2} &= \frac{\partial \sigma_{11}}{\partial x_1} + \frac{\partial \sigma_{12}}{\partial x_2} + \frac{\partial \sigma_{13}}{\partial x_3} + f_1 \nonumber \\
\rho\frac{\partial^2 u_2}{\partial t^2} &= \frac{\partial \sigma_{21}}{\partial x_1} + \frac{\partial \sigma_{22}}{\partial x_2} + \frac{\partial \sigma_{23}}{\partial x_3} + f_2 \nonumber \\
\rho\frac{\partial^2 u_3}{\partial t^2} &= \frac{\partial \sigma_{31}}{\partial x_1} + \frac{\partial \sigma_{32}}{\partial x_2} + \frac{\partial \sigma_{33}}{\partial x_3} + f_3 \nonumber \\
\sigma_{11} &= \lambda(\epsilon_{11}+\epsilon_{22}+\epsilon_{33}) + 2 \mu \epsilon_{11}\nonumber \\
\sigma_{22} &= \lambda(\epsilon_{11}+\epsilon_{22}+\epsilon_{33}) + 2 \mu \epsilon_{22}\nonumber \\
\sigma_{33} &= \lambda(\epsilon_{11}+\epsilon_{22}+\epsilon_{33}) + 2 \mu \epsilon_{33}\nonumber \\
\sigma_{12} &= 2 \mu \epsilon_{12}\nonumber \\
\sigma_{13} &= 2 \mu \epsilon_{13}\nonumber \\
\sigma_{23} &= 2 \mu \epsilon_{23}\nonumber \\
\end{align}


## We learned:

* 3 different approaches to derive the stress-strain relation for an isotropic elastic medium

  1. Manually, by straightforward computation of the stress tensor components
  2. Using the symbolic math library **SymPy**
  3. General evaluation of the stress-strain relation and neglecting non-zero contributions
* 3D equations of motion for isotropic elastic medium

## References

1. Hodge, P.G. (1961) "[On Isotropic Cartesian Tensors](https://www.jstor.org/stable/2311997?seq=1#page_scan_tab_contents)", The American Mathematical Monthly, 68(8), 793-95.

2. Jeffreys, H. (1969) "[Cartesian Tensors](https://archive.org/details/CartesianTensors)", Cambridge University Press
